In [1]:
'''
This file plots CMIP5 RCP - HIST over South Australia
/Users/earl/Desktop/Yang
and places the outputs in
/Users/earl/Dropbox/CMIP5/figures

Earl Duran 
created: 19-Mar-18
e.duran@unsw.edu.au
'''

import pandas as pd
import cosima_cookbook as cc
import os
import xarray as xr
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib
import sys
import pickle
import itertools
def find_nearest_index(array, value):
    return int((np.abs(array - value)).argmin())
from scipy import interpolate
from scipy import stats
from dask.distributed import Client

import cartopy.crs as ccrs
import cartopy.feature as cft
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
def arange(start,end,stride=1):
    return np.arange(start,end+0.00000001,stride)

import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata

session = cc.database.create_session()

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
client = Client()
client

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:41500 Dashboard: http://127.0.0.1:34976/status,Cluster Workers: 4 Cores: 8 Memory: 33.67 GB


In [4]:
output_path = '/g/data/e14/erd561/Australia_All/'

In [5]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

net_sfc_heating_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='net_sfc_heating').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(net_sfc_heating_1_mod_month)

net_sfc_heating_1_mod_month.load()
print(net_sfc_heating_1_mod_month)

xt_ocean = net_sfc_heating_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
net_sfc_heating_1_mod_month = net_sfc_heating_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(net_sfc_heating_1_mod_month)

net_sfc_heating_1_mod = net_sfc_heating_1_mod_month.groupby('time.year').mean('time')
print(net_sfc_heating_1_mod)

net_sfc_heating_1_mod = net_sfc_heating_1_mod.sel(year=slice(1993+60*4,2017+60*4))
print(net_sfc_heating_1_mod)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'net_sfc_heating' (time: 3600, yt_ocean: 160, xt_ocean: 90)>
dask.array<getitem, shape=(3600, 160, 90), dtype=float32, chunksize=(1, 160, 90), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      surface ocean heat flux coming through coupler and mass t...
    units:          Watts/m^2
    valid_range:    [-10000.  10000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
<xarray.DataArray 'net_sfc_heating' (time: 3600, yt_ocean: 160, xt_ocean: 90)>
array([[[          nan,           nan,           nan, ...,
           89.22634   ,   86.25432   ,   84.09215   ],
        [          nan,           nan,           nan, ...,
           90.62474   ,   86.926735  , 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'net_sfc_heating' (year: 300, yt_ocean: 160, xt_ocean: 90)>
array([[[            nan,             nan,             nan, ...,
         -5.08226929e+01, -4.69555397e+01, -4.42211418e+01],
        [            nan,             nan,             nan, ...,
         -4.75538902e+01, -4.83445282e+01, -4.59344215e+01],
        [            nan,             nan,             nan, ...,
         -4.48273582e+01, -4.46863289e+01, -4.39777412e+01],
        ...,
        [ 4.27722025e+00,  5.39699888e+00,  8.58406067e-01, ...,
         -1.15573730e+01, -1.30522947e+01, -1.41863184e+01],
        [ 5.21874285e+00,  2.21401906e+00, -1.47089517e+00, ...,
         -1.21350222e+01, -1.40236158e+01, -1.51137619e+01],
        [ 4.30260229e+00,  6.37573242e-01, -6.65490389e+00, ...,
         -1.16685638e+01, -1.23913803e+01, -1.37005148e+01]],

       [[            nan,             nan,             nan, ...,
         -4.66988525e+01, -4.59515190e+01, -4.54492836e+01],
        [            nan,

In [ ]:
lat = net_sfc_heating_1_mod.yt_ocean
lon = net_sfc_heating_1_mod.xt_ocean
year = net_sfc_heating_1_mod.year

# net_sfc_heating_1_mod_trans = np.transpose(net_sfc_heating_1_mod, (2,1,0))
net_sfc_heating_1_mod_trans = net_sfc_heating_1_mod
print(np.shape(net_sfc_heating_1_mod_trans))
net_sfc_heating_1_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_1_mod')
net_sfc_heating_1_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_1_mod')
net_sfc_heating_1_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_1_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        net_sfc_heating_1_mod_slope[iid, jid], _, _, net_sfc_heating_1_mod_p_value[iid, jid], net_sfc_heating_1_mod_std_err[iid, jid] = \
        stats.linregress(year, net_sfc_heating_1_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(net_sfc_heating_1_mod_slope)

(25, 160, 90)
lat -69.6292858330716
lat -69.17632154294074
lat -68.70935410409521
lat -68.22843658249317
lat -67.73301847540898
lat -67.2232002021303
lat -66.69847936238045
lat -66.15900514695375
lat -65.60432451496254
lat -65.03463652231858
lat -64.44953841598465
lat -63.84927987875662
lat -63.233509039251814
lat -62.602526632003375
lat -61.95603192247815
lat -61.29437678205883
lat -60.61731152794955
lat -59.92523891318751
lat -59.21795988186098
lat -58.49592747485757
lat -57.75899250138302
lat -57.00765736171395
lat -56.24182163656275
lat -55.46203582865502
lat -54.66824687203264
lat -53.86105179481633
lat -53.04044315107174
lat -52.20706260764199
lat -51.36094630171701
lat -50.50277835512628
lat -49.63263616124607
lat -48.751243830625256
lat -47.858717411335334
lat -46.955818270259286
lat -46.04269825143213
lat -45.12015299775091
lat -44.1883670522662
lat -43.24816712546786
lat -42.299767144867545
lat -41.34402147337693
lat -40.381169912865516
lat -39.412090879473524
lat -38.4370463

In [ ]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

net_sfc_heating_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='net_sfc_heating').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(net_sfc_heating_025_mod_month)

net_sfc_heating_025_mod_month.load()
print(net_sfc_heating_025_mod_month)

xt_ocean = net_sfc_heating_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
net_sfc_heating_025_mod_month = net_sfc_heating_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(net_sfc_heating_025_mod_month)

net_sfc_heating_025_mod = net_sfc_heating_025_mod_month.groupby('time.year').mean('time')
print(net_sfc_heating_025_mod)

net_sfc_heating_025_mod = net_sfc_heating_025_mod.sel(year=slice(1993+60*4,2017+60*4))
print(net_sfc_heating_025_mod)

In [ ]:
lat = net_sfc_heating_025_mod.yt_ocean
lon = net_sfc_heating_025_mod.xt_ocean
year = net_sfc_heating_025_mod.year

# net_sfc_heating_025_mod_trans = np.transpose(net_sfc_heating_025_mod, (2,1,0))
net_sfc_heating_025_mod_trans = net_sfc_heating_025_mod
print(np.shape(net_sfc_heating_025_mod_trans))
net_sfc_heating_025_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_025_mod')
net_sfc_heating_025_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_025_mod')
net_sfc_heating_025_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_025_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        net_sfc_heating_025_mod_slope[iid, jid], _, _, net_sfc_heating_025_mod_p_value[iid, jid], net_sfc_heating_025_mod_std_err[iid, jid] = \
        stats.linregress(year, net_sfc_heating_025_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(net_sfc_heating_025_mod_slope)

In [ ]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

net_sfc_heating_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='net_sfc_heating').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(net_sfc_heating_01_mod_month)

net_sfc_heating_01_mod_month.load()
print(net_sfc_heating_01_mod_month)

xt_ocean = net_sfc_heating_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
net_sfc_heating_01_mod_month = net_sfc_heating_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(net_sfc_heating_01_mod_month)

net_sfc_heating_01_mod = net_sfc_heating_01_mod_month.groupby('time.year').mean('time')
print(net_sfc_heating_01_mod)

net_sfc_heating_01_mod = net_sfc_heating_01_mod.sel(year=slice(1993,2017))
print(net_sfc_heating_01_mod)

In [ ]:
lat = net_sfc_heating_01_mod.yt_ocean
lon = net_sfc_heating_01_mod.xt_ocean
year = net_sfc_heating_01_mod.year

# net_sfc_heating_01_mod_trans = np.transpose(net_sfc_heating_01_mod, (2,1,0))
net_sfc_heating_01_mod_trans = net_sfc_heating_01_mod
print(np.shape(net_sfc_heating_01_mod_trans))
net_sfc_heating_01_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_01_mod')
net_sfc_heating_01_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_01_mod')
net_sfc_heating_01_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='net_sfc_heating_01_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        net_sfc_heating_01_mod_slope[iid, jid], _, _, net_sfc_heating_01_mod_p_value[iid, jid], net_sfc_heating_01_mod_std_err[iid, jid] = \
        stats.linregress(year, net_sfc_heating_01_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(net_sfc_heating_01_mod_slope)

In [17]:
net_sfc_heating_1_mod_xr = xr.DataArray(net_sfc_heating_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='net_sfc_heating_1_mod',
                        coords=[net_sfc_heating_1_mod.yt_ocean, net_sfc_heating_1_mod.xt_ocean, net_sfc_heating_1_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
net_sfc_heating_1_mod_xr.to_netcdf(output_path + 'net_sfc_heating_1_mod.nc')


net_sfc_heating_025_mod_xr = xr.DataArray(net_sfc_heating_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='net_sfc_heating_025_mod',
                        coords=[net_sfc_heating_025_mod.yt_ocean, net_sfc_heating_025_mod.xt_ocean, net_sfc_heating_025_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
net_sfc_heating_025_mod_xr.to_netcdf(output_path + 'net_sfc_heating_025_mod.nc')


net_sfc_heating_01_mod_xr = xr.DataArray(net_sfc_heating_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='net_sfc_heating_01_mod',
                        coords=[net_sfc_heating_01_mod.yt_ocean, net_sfc_heating_01_mod.xt_ocean, net_sfc_heating_01_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
net_sfc_heating_01_mod_xr.to_netcdf(output_path + 'net_sfc_heating_01_mod.nc')



net_sfc_heating_1_mod_slope_xr = xr.DataArray(net_sfc_heating_1_mod_slope, name='net_sfc_heating_1_mod_slope',
                        coords=[net_sfc_heating_1_mod_slope.lat, net_sfc_heating_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_1_mod_slope_xr.to_netcdf(output_path + 'net_sfc_heating_1_mod_slope.nc')

net_sfc_heating_1_mod_p_value_xr = xr.DataArray(net_sfc_heating_1_mod_p_value, name='net_sfc_heating_1_mod_p_value',
                        coords=[net_sfc_heating_1_mod_slope.lat, net_sfc_heating_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_1_mod_p_value_xr.to_netcdf(output_path + 'net_sfc_heating_1_mod_p_value.nc')

net_sfc_heating_1_mod_std_err_xr = xr.DataArray(net_sfc_heating_1_mod_std_err, name='net_sfc_heating_1_mod_std_err',
                        coords=[net_sfc_heating_1_mod_slope.lat, net_sfc_heating_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_1_mod_std_err_xr.to_netcdf(output_path + 'net_sfc_heating_1_mod_std_err.nc')



net_sfc_heating_025_mod_slope_xr = xr.DataArray(net_sfc_heating_025_mod_slope, name='net_sfc_heating_025_mod_slope',
                        coords=[net_sfc_heating_025_mod_slope.lat, net_sfc_heating_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_025_mod_slope_xr.to_netcdf(output_path + 'net_sfc_heating_025_mod_slope.nc')

net_sfc_heating_025_mod_p_value_xr = xr.DataArray(net_sfc_heating_025_mod_p_value, name='net_sfc_heating_025_mod_p_value',
                        coords=[net_sfc_heating_025_mod_slope.lat, net_sfc_heating_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_025_mod_p_value_xr.to_netcdf(output_path + 'net_sfc_heating_025_mod_p_value.nc')

net_sfc_heating_025_mod_std_err_xr = xr.DataArray(net_sfc_heating_025_mod_std_err, name='net_sfc_heating_025_mod_std_err',
                        coords=[net_sfc_heating_025_mod_slope.lat, net_sfc_heating_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_025_mod_std_err_xr.to_netcdf(output_path + 'net_sfc_heating_025_mod_std_err.nc')



net_sfc_heating_01_mod_slope_xr = xr.DataArray(net_sfc_heating_01_mod_slope, name='net_sfc_heating_01_mod_slope',
                        coords=[net_sfc_heating_01_mod_slope.lat, net_sfc_heating_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_01_mod_slope_xr.to_netcdf(output_path + 'net_sfc_heating_01_mod_slope.nc')

net_sfc_heating_01_mod_p_value_xr = xr.DataArray(net_sfc_heating_01_mod_p_value, name='net_sfc_heating_01_mod_p_value',
                        coords=[net_sfc_heating_01_mod_slope.lat, net_sfc_heating_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_01_mod_p_value_xr.to_netcdf(output_path + 'net_sfc_heating_01_mod_p_value.nc')

net_sfc_heating_01_mod_std_err_xr = xr.DataArray(net_sfc_heating_01_mod_std_err, name='net_sfc_heating_01_mod_std_err',
                        coords=[net_sfc_heating_01_mod_slope.lat, net_sfc_heating_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
net_sfc_heating_01_mod_std_err_xr.to_netcdf(output_path + 'net_sfc_heating_01_mod_std_err.nc')

In [6]:
latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_1_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(pme_river_1_mod_month)

pme_river_1_mod_month.load()
print(pme_river_1_mod_month)

xt_ocean = pme_river_1_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_1_mod_month = pme_river_1_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_1_mod_month)

pme_river_1_mod = pme_river_1_mod_month.groupby('time.year').mean('time')
print(pme_river_1_mod)

pme_river_1_mod = pme_river_1_mod.sel(year=slice(2004+60*4,2017+60*4))
print(pme_river_1_mod)

lat = pme_river_1_mod.yt_ocean
lon = pme_river_1_mod.xt_ocean
year = pme_river_1_mod.year

# pme_river_1_mod_trans = np.transpose(pme_river_1_mod, (2,1,0))
pme_river_1_mod_trans = pme_river_1_mod
print(np.shape(pme_river_1_mod_trans))
pme_river_1_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_1_mod')
pme_river_1_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_1_mod')
pme_river_1_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_1_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        pme_river_1_mod_slope[iid, jid], _, _, pme_river_1_mod_p_value[iid, jid], pme_river_1_mod_std_err[iid, jid] = \
        stats.linregress(year, pme_river_1_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(pme_river_1_mod_slope)

latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_025_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(pme_river_025_mod_month)

pme_river_025_mod_month.load()
print(pme_river_025_mod_month)

xt_ocean = pme_river_025_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_025_mod_month = pme_river_025_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_025_mod_month)

pme_river_025_mod = pme_river_025_mod_month.groupby('time.year').mean('time')
print(pme_river_025_mod)

pme_river_025_mod = pme_river_025_mod.sel(year=slice(2004+60*4,2017+60*4))
print(pme_river_025_mod)

lat = pme_river_025_mod.yt_ocean
lon = pme_river_025_mod.xt_ocean
year = pme_river_025_mod.year

# pme_river_025_mod_trans = np.transpose(pme_river_025_mod, (2,1,0))
pme_river_025_mod_trans = pme_river_025_mod
print(np.shape(pme_river_025_mod_trans))
pme_river_025_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_025_mod')
pme_river_025_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_025_mod')
pme_river_025_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_025_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        pme_river_025_mod_slope[iid, jid], _, _, pme_river_025_mod_p_value[iid, jid], pme_river_025_mod_std_err[iid, jid] = \
        stats.linregress(year, pme_river_025_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(pme_river_025_mod_slope)

latS = -70
latN = 20
lonW = 90-360
lonE = 180-360
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
print(expt)

pme_river_01_mod_month = cc.querying.getvar(expt=expt,
                                     session=session,
                                     ncfile='ocean_month.nc',
                                     variable='pme_river').\
sel(yt_ocean=slice(latS,latN)).\
sel(xt_ocean=slice(lonW,lonE))
print(pme_river_01_mod_month)

pme_river_01_mod_month.load()
print(pme_river_01_mod_month)

xt_ocean = pme_river_01_mod_month.xt_ocean
xt_ocean_corrected = xt_ocean + 360
pme_river_01_mod_month = pme_river_01_mod_month.assign_coords(xt_ocean=xt_ocean_corrected)
print(pme_river_01_mod_month)

pme_river_01_mod = pme_river_01_mod_month.groupby('time.year').mean('time')
print(pme_river_01_mod)

pme_river_01_mod = pme_river_01_mod.sel(year=slice(2004,2017))
print(pme_river_01_mod)

lat = pme_river_01_mod.yt_ocean
lon = pme_river_01_mod.xt_ocean
year = pme_river_01_mod.year

# pme_river_01_mod_trans = np.transpose(pme_river_01_mod, (2,1,0))
pme_river_01_mod_trans = pme_river_01_mod
print(np.shape(pme_river_01_mod_trans))
pme_river_01_mod_slope = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_01_mod')
pme_river_01_mod_p_value = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_01_mod')
pme_river_01_mod_std_err = xr.DataArray(
    np.zeros([np.shape(lat)[0], np.shape(lon)[0]]), dims=(
    'lat', 'lon'), coords=[lat, lon], name='pme_river_01_mod')
for iid, i in enumerate(lat):
    for jid, j in enumerate(lon):
        pme_river_01_mod_slope[iid, jid], _, _, pme_river_01_mod_p_value[iid, jid], pme_river_01_mod_std_err[iid, jid] = \
        stats.linregress(year, pme_river_01_mod_trans[:, iid, jid])
    print('lat ' + str(np.array(i)))
print(pme_river_01_mod_slope)

1deg_jra55v13_iaf_spinup1_B1


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/cosima_cookbook/querying.py:96: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  preprocess=lambda d: d[variable].to_dataset() if variable not in d.coords else d)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/backends/api.py:934: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option t

<xarray.DataArray 'pme_river' (time: 3600, yt_ocean: 160, xt_ocean: 90)>
dask.array<getitem, shape=(3600, 160, 90), dtype=float32, chunksize=(1, 160, 90), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -69.63 -69.18 -68.71 ... 17.85 18.59 19.36
  * xt_ocean  (xt_ocean) float64 -269.5 -268.5 -267.5 ... -182.5 -181.5 -180.5
  * time      (time) object 1958-01-16 12:00:00 ... 2257-12-16 12:00:00
Attributes:
    long_name:      mass flux of precip-evap+river via sbc (liquid, frozen, e...
    units:          (kg/m^3)*(m/sec)
    valid_range:    [-1000000.  1000000.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t
    standard_name:  water_flux_into_sea_water
<xarray.DataArray 'pme_river' (time: 3600, yt_ocean: 160, xt_ocean: 90)>
array([[[            nan,             nan,             nan, ...,
          1.42368299e-05,  1.44414416e-05,  1.34372858e-05],
        [            nan,             nan

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.07/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.DataArray 'pme_river' (year: 300, yt_ocean: 160, xt_ocean: 90)>
array([[[            nan,             nan,             nan, ...,
          2.10337439e-05,  1.49958432e-05,  7.74307500e-06],
        [            nan,             nan,             nan, ...,
          1.62638771e-05,  1.79659401e-05,  1.05090485e-05],
        [            nan,             nan,             nan, ...,
          1.20858995e-05,  1.17251220e-05,  1.13022252e-05],
        ...,
        [ 2.94816637e-05,  2.75560105e-05,  2.43981558e-05, ...,
         -4.79396149e-05, -4.85968085e-05, -4.85991222e-05],
        [ 3.11815274e-05,  2.73534242e-05,  2.23394891e-05, ...,
         -4.81069474e-05, -4.93650623e-05, -4.99544149e-05],
        [ 2.81702432e-05,  2.25848707e-05,  1.40597222e-05, ...,
         -4.80191484e-05, -4.84307820e-05, -4.89228296e-05]],

       [[            nan,             nan,             nan, ...,
          4.12223380e-06,  7.48361890e-06,  1.03621524e-05],
        [            nan,      

...

<xarray.DataArray 'pme_river' (year: 33, yt_ocean: 1186, xt_ocean: 900)>
array([[[           nan,            nan,            nan, ...,
          2.3295872e-05,  2.4080584e-05,  2.3201028e-05],
        [           nan,            nan,            nan, ...,
          1.7924243e-05,  1.9243227e-05,  2.0672500e-05],
        [           nan,            nan,            nan, ...,
          1.1768229e-05,  1.4892612e-05,  1.7631002e-05],
        ...,
        [ 1.5243183e-05,  1.5309752e-05,  1.5162953e-05, ...,
         -3.9551571e-05, -3.9466962e-05, -3.9477691e-05],
        [ 1.5490950e-05,  1.5501560e-05,  1.5372412e-05, ...,
         -3.9556708e-05, -3.9536913e-05, -3.9517268e-05],
        [ 1.5895946e-05,  1.5961239e-05,  1.5975631e-05, ...,
         -3.9546547e-05, -3.9579314e-05, -3.9552560e-05]],

       [[           nan,            nan,            nan, ...,
          3.6079204e-05,  3.4885634e-05,  3.3777993e-05],
        [           nan,            nan,            nan, ...,
          

lat -69.99968147628742
lat -69.95744212190694
lat -69.91520276752645
lat -69.87296341314597
lat -69.83072405876548
lat -69.788484704385
lat -69.74624535000451
lat -69.70400599562403
lat -69.66176664124355
lat -69.61952728686306
lat -69.57728793248258
lat -69.53504857810209
lat -69.49280922372161
lat -69.45056986934112
lat -69.40833051496064
lat -69.36609116058015
lat -69.32385180619967
lat -69.28161245181919
lat -69.2393730974387
lat -69.19713374305822
lat -69.15489438867773
lat -69.11265503429725
lat -69.07041567991676
lat -69.02817632553628
lat -68.9859369711558
lat -68.94369761677531
lat -68.90145826239483
lat -68.85921890801434
lat -68.81697955363386
lat -68.77474019925337
lat -68.73250084487289
lat -68.6902614904924
lat -68.64802213611192
lat -68.60578278173143
lat -68.56354342735095
lat -68.52130407297047
lat -68.47906471858998
lat -68.4368253642095
lat -68.39458600982901
lat -68.35234665544853
lat -68.31010730106804
lat -68.26786794668756
lat -68.22562859230707
lat -68.183389237

lat -52.8919458519512
lat -52.831571790885135
lat -52.77111382997932
lat -52.710571667773955
lat -52.649945506616675
lat -52.589235046361054
lat -52.52844048927524
lat -52.46756153653012
lat -52.40659839032443
lat -52.34555075315034
lat -52.28441882714726
lat -52.22320231613265
lat -52.16190142219672
lat -52.100515850486254
lat -52.03904580305239
lat -51.97749098637534
lat -51.915851602477304
lat -51.85412735917605
lat -51.79231845847502
lat -51.73042460953369
lat -51.66844601434695
lat -51.606382383420154
lat -51.54423391874985
lat -51.48200033219155
lat -51.41968182575365
lat -51.357278112646064
lat -51.29478939489933
lat -51.232215387082064
lat -51.169556291257194
lat -51.106811823356374
lat -51.04398218548522
lat -50.981067094942794
lat -50.91806675388769
lat -50.8549808809908
lat -50.79180967847401
lat -50.728552866384454
lat -50.665210647017666
lat -50.60178274180151
lat -50.53826935311551
lat -50.47467020377275
lat -50.41098549624713
lat -50.347214954741474
lat -50.2833587818344

lat -26.277227333576807
lat -26.18752648802262
lat -26.097756440389567
lat -26.007917527624393
lat -25.918009748278212
lat -25.828033440650042
lat -25.737988605161227
lat -25.647875581463406
lat -25.557694371844295
lat -25.467445317308336
lat -25.377128422005594
lat -25.28674402829334
lat -25.19629214217983
lat -25.105773107375004
lat -25.015186931740978
lat -24.924533960340074
lat -24.833814202883772
lat -24.743028005786304
lat -24.652175380603865
lat -24.56125667510197
lat -24.47027190267672
lat -24.37922141244411
lat -24.288105219635188
lat -24.196923674715446
lat -24.10567679474576
lat -24.014364931539998
lat -23.922988103983556
lat -23.83154666523737
lat -23.74004063600592
lat -23.64847037079573
lat -23.556835892124752
lat -23.465137555843462
lat -23.373375386277488
lat -23.281549740619457
lat -23.189660644996728
lat -23.097708457942094
lat -23.005693207378542
lat -22.913615253176868
lat -22.82147462504942
lat -22.729271684202672
lat -22.637006462131883
lat -22.54467932137672
lat 

lat 8.122658501262137
lat 8.221643042230964
lat 8.32060295792327
lat 8.419537805456535
lat 8.51844745017845
lat 8.617331449874465
lat 8.716189670725734
lat 8.815021671199979
lat 8.913827318329801
lat 9.012606171279373
lat 9.111358097950628
lat 9.210082658218257
lat 9.308779720871298
lat 9.407448846508915
lat 9.506089904824908
lat 9.604702457156753
lat 9.703286374120554
lat 9.801841217805821
lat 9.900366859768386
lat 9.998862862863398
lat 10.09732909960372
lat 10.195765133623615
lat 10.294170838410169
lat 10.392545778390115
lat 10.490889828041823
lat 10.589202552597722
lat 10.687483827544405
lat 10.785733218933103
lat 10.883950603275439
lat 10.982135547454417
lat 11.080287929023381
lat 11.178407315709944
lat 11.27649358612572
lat 11.374546308855647
lat 11.47256536358603
lat 11.570550319771696
lat 11.668501058189939
lat 11.766417149177911
lat 11.864298474620044
lat 11.962144605748115
lat 12.059955425569713
lat 12.157730506223396
lat 12.25546973185579
lat 12.35317267552425
lat 12.45083922

In [7]:
pme_river_1_mod_xr = xr.DataArray(pme_river_1_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_1_mod',
                        coords=[pme_river_1_mod.yt_ocean, pme_river_1_mod.xt_ocean, pme_river_1_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
pme_river_1_mod_xr.to_netcdf(output_path + 'pme_river_1_mod.nc')


pme_river_025_mod_xr = xr.DataArray(pme_river_025_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_025_mod',
                        coords=[pme_river_025_mod.yt_ocean, pme_river_025_mod.xt_ocean, pme_river_025_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
pme_river_025_mod_xr.to_netcdf(output_path + 'pme_river_025_mod.nc')


pme_river_01_mod_xr = xr.DataArray(pme_river_01_mod.transpose('yt_ocean', 'xt_ocean', 'year'), name='pme_river_01_mod',
                        coords=[pme_river_01_mod.yt_ocean, pme_river_01_mod.xt_ocean, pme_river_01_mod.year], 
                        dims=['yt_ocean', 'xt_ocean', 'year'])
pme_river_01_mod_xr.to_netcdf(output_path + 'pme_river_01_mod.nc')



pme_river_1_mod_slope_xr = xr.DataArray(pme_river_1_mod_slope, name='pme_river_1_mod_slope',
                        coords=[pme_river_1_mod_slope.lat, pme_river_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_1_mod_slope_xr.to_netcdf(output_path + 'pme_river_1_mod_slope.nc')

pme_river_1_mod_p_value_xr = xr.DataArray(pme_river_1_mod_p_value, name='pme_river_1_mod_p_value',
                        coords=[pme_river_1_mod_slope.lat, pme_river_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_1_mod_p_value_xr.to_netcdf(output_path + 'pme_river_1_mod_p_value.nc')

pme_river_1_mod_std_err_xr = xr.DataArray(pme_river_1_mod_std_err, name='pme_river_1_mod_std_err',
                        coords=[pme_river_1_mod_slope.lat, pme_river_1_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_1_mod_std_err_xr.to_netcdf(output_path + 'pme_river_1_mod_std_err.nc')



pme_river_025_mod_slope_xr = xr.DataArray(pme_river_025_mod_slope, name='pme_river_025_mod_slope',
                        coords=[pme_river_025_mod_slope.lat, pme_river_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_025_mod_slope_xr.to_netcdf(output_path + 'pme_river_025_mod_slope.nc')

pme_river_025_mod_p_value_xr = xr.DataArray(pme_river_025_mod_p_value, name='pme_river_025_mod_p_value',
                        coords=[pme_river_025_mod_slope.lat, pme_river_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_025_mod_p_value_xr.to_netcdf(output_path + 'pme_river_025_mod_p_value.nc')

pme_river_025_mod_std_err_xr = xr.DataArray(pme_river_025_mod_std_err, name='pme_river_025_mod_std_err',
                        coords=[pme_river_025_mod_slope.lat, pme_river_025_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_025_mod_std_err_xr.to_netcdf(output_path + 'pme_river_025_mod_std_err.nc')



pme_river_01_mod_slope_xr = xr.DataArray(pme_river_01_mod_slope, name='pme_river_01_mod_slope',
                        coords=[pme_river_01_mod_slope.lat, pme_river_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_01_mod_slope_xr.to_netcdf(output_path + 'pme_river_01_mod_slope.nc')

pme_river_01_mod_p_value_xr = xr.DataArray(pme_river_01_mod_p_value, name='pme_river_01_mod_p_value',
                        coords=[pme_river_01_mod_slope.lat, pme_river_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_01_mod_p_value_xr.to_netcdf(output_path + 'pme_river_01_mod_p_value.nc')

pme_river_01_mod_std_err_xr = xr.DataArray(pme_river_01_mod_std_err, name='pme_river_01_mod_std_err',
                        coords=[pme_river_01_mod_slope.lat, pme_river_01_mod_slope.lon], 
                        dims=['yt_ocean', 'xt_ocean'])
pme_river_01_mod_std_err_xr.to_netcdf(output_path + 'pme_river_01_mod_std_err.nc')